## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [ ]:
# Your code here - remember to use markdown cells for comments as well!

In [2]:
import gzip
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [1]:
!ls 'zippedData'

bom.movie_gross.csv
imdb.name.basics.csv
imdb.title.akas.csv
imdb.title.basics.csv
imdb.title.crew.csv
imdb.title.principals.csv
imdb.title.ratings.csv
rt.movie_info.tsv
rt.reviews.tsv
tmdb.movies.csv
tn.movie_budgets.csv


In [6]:
#Questions that I am looking to answer: 
#When is the best time of year to release films?
#Is there a correlation between film genres and their release dates?
#How successful (profitable) are film releases during particular quarters?

#store all csv/tsv files as dataframes in order to examine
movies_gross = pd.read_csv('zippedData/bom.movie_gross.csv')
name_basics = pd.read_csv('zippedData/imdb.name.basics.csv')
title_akas = pd.read_csv('zippedData/imdb.title.akas.csv')
title_basics = pd.read_csv('zippedData/imdb.title.basics.csv')
title_crew = pd.read_csv('zippedData/imdb.title.crew.csv')
title_principals = pd.read_csv('zippedData/imdb.title.crew.csv')
title_ratings = pd.read_csv('zippedData/imdb.title.ratings.csv')
movie_info = pd.read_csv('zippedData/rt.movie_info.tsv', sep = '\t', encoding = 'unicode_escape')
reviews = pd.read_csv('zippedData/rt.reviews.tsv', sep = '\t', encoding = 'unicode_escape')
movies = pd.read_csv('zippedData/tmdb.movies.csv')
movie_budgets = pd.read_csv('zippedData/tn.movie_budgets.csv')

In [9]:
#display basic info for all dataframes to decide which metrics to use

print(movies_gross.info())
print(name_basics.info())
print(title_akas.info())
print(title_basics.info())
print(title_crew.info())
print(title_principals.info())
print(title_ratings.info())
print(movie_info.info())
print(reviews.info())
print(movies.info())
print(movie_budgets.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606648 entries, 0 to 606647
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   nconst              606648 non-null  object 
 1   primary_name        606648 non-null  object 
 2   birth_year          82736 non-null   float64
 3   death_year          6783 non-null    float64
 4   primary_profession  555308 non-null  object 
 5   known_for_titles    576444 non-null  object 


A list of potentially useful metrics to answer my questions:

from movies_gross:
title, domestic_gross, foreign_gross, year

from title_akas:
title_id, title

from title_basics:
primary_title, start_year, genres

title_ratings:
tconst, averagerating, numvotes

from movie_info:
id, rating, genre, theater_date, box_office?

from reviews:
id, rating

from movies:
genre_ids, id, popularity, release_date, title, vote_average

from movie_budgets:
id, release_date, movie, domestic_gross, worldwide_gross

In [54]:
movies_gross_selected = movies_gross[['title', 'domestic_gross', 'foreign_gross', 'year']]


movies_gross_selected.sort_values('year')

,title,domestic_gross,foreign_gross,year
0,Toy Story 3,415000000.0,652000000,2010
222,Flipped,1800000.0,NaN,2010
221,Cairo Time,1600000.0,391000,2010
220,After.Life,109000.0,1900000,2010
219,Golmaal 3,1100000.0,935000,2010
...,...,...,...,...
3177,Mary Queen of Scots,16500000.0,29900000,2018
3176,Tyler Perry's Acrimony,43500000.0,2900000,2018
3175,Uncle Drew,42500000.0,4200000,2018
3182,Show Dogs,17900000.0,21300000,2018


In [55]:
title_basics_selected = title_basics[['primary_title', 'start_year', 'genres']]

title_basics_selected.sort_values('start_year')

,primary_title,start_year,genres
9599,Brainiacs in La La Land,2010,Comedy
43264,Fireplace for your Home: Crackling Fireplace w...,2010,Music
11550,Role/Play,2010,"Drama,Romance"
11551,Johan1,2010,"Comedy,Drama,Romance"
11552,Hands Up,2010,Drama
...,...,...,...
2948,Untitled Star Wars Film,2024,NaN
52213,Avatar 4,2025,"Action,Adventure,Fantasy"
2949,Untitled Star Wars Film,2026,Fantasy
96592,Avatar 5,2027,"Action,Adventure,Fantasy"


In [51]:
title_akas_selected = title_akas[['title_id', 'title']]

title_akas_selected

,title_id,title
0,tt0369610,Джурасик свят
1,tt0369610,Jurashikku warudo
2,tt0369610,Jurassic World: O Mundo dos Dinossauros
3,tt0369610,O Mundo dos Dinossauros
4,tt0369610,Jurassic World
...,...,...
331698,tt9827784,Sayonara kuchibiru
331699,tt9827784,Farewell Song
331700,tt9880178,La atención
331701,tt9880178,La atención


In [56]:
title_ratings_selected = title_ratings[['tconst', 'averagerating', 'numvotes']]

title_ratings_selected.sort_values('averagerating', ascending = False)

,tconst,averagerating,numvotes
27644,tt8730716,10.0,5
8992,tt1770682,10.0,5
26153,tt2632430,10.0,5
65033,tt9715646,10.0,20
57679,tt5344358,10.0,6
...,...,...,...
50971,tt6017238,1.0,21
32440,tt2087999,1.0,7
36456,tt1896768,1.0,21
54856,tt1976600,1.0,436


In [58]:
movie_info_selected = movie_info[['id', 'rating', 'genre', 'theater_date', 'box_office']]


movie_info_selected.sort_values('theater_date', ascending = False)

type(movie_info_selected)

,id,rating,genre,theater_date,box_office
716,921,PG-13,Action and Adventure|Comedy|Drama|Mystery and ...,"Sep 9, 1992",NaN
1374,1773,PG-13,Comedy,"Sep 9, 1988",NaN
715,919,NR,Drama,"Sep 9, 1986",NaN
1356,1751,R,Comedy|Drama,"Sep 8, 2000",NaN
1434,1850,PG-13,Drama,"Sep 8, 1996",NaN
...,...,...,...,...,...
1543,1982,NaN,NaN,NaN,NaN
1547,1987,PG-13,Comedy,NaN,NaN
1548,1988,R,Comedy,NaN,NaN
1549,1989,NR,Art House and International|Drama,NaN,NaN


In [40]:
reviews_selected = reviews[['id', 'rating']]

In [59]:
movies_selected = movies[['genre_ids', 'id', 'popularity', 'release_date', 'title', 'vote_average']]
movies_selected.sort_values('vote_average', ascending = False)

,genre_ids,id,popularity,release_date,title,vote_average
13258,[99],403294,0.600,2014-07-04,9/11: Simulations,10.0
22787,"[27, 35]",444170,0.722,2017-02-23,Lasagna Cat - Telephone Sex Survey Results,10.0
22592,[99],488357,0.900,2017-11-21,MOTO 9: The Movie,10.0
22607,[],437448,0.895,2017-01-27,The Letter Carrier,10.0
22610,"[9648, 37, 18, 10402]",463601,0.894,2017-06-26,Hope This Goes Over Well/Living in an Empty World,10.0
...,...,...,...,...,...,...
11018,[],124727,0.600,2013-08-09,Darkchylde,0.0
14168,[],167531,0.600,2014-07-25,Summer Snow,0.0
14167,"[99, 10402]",221918,0.600,2014-08-05,Butch Walker: Out of Focus,0.0
20601,"[18, 35]",448105,0.600,2017-04-08,The Angry Man,0.0


In [25]:
type(movies['release_date'][0])

str

In [62]:
movie_budgets_selected = movie_budgets[['id', 'release_date', 'movie', 'domestic_gross', 'worldwide_gross']]

movie_budgets_selected.sort_values('worldwide_gross', ascending = False)

movie_budgets_selected['cleaned_ww_gross'] = []

,id,release_date,movie,domestic_gross,worldwide_gross
3737,38,"Aug 21, 2009",Fifty Dead Men Walking,$0,"$997,921"
3432,33,"Sep 30, 2005",Duma,"$870,067","$994,790"
5062,63,"Apr 1, 2011",Insidious,"$54,009,150","$99,870,886"
883,84,"Apr 2, 2004",Hellboy,"$59,623,958","$99,823,958"
5613,14,"Mar 21, 1980",Mad Max,"$8,750,000","$99,750,000"
...,...,...,...,...,...
5488,89,"Dec 31, 2014",The Sound and the Shadow,$0,$0
5487,88,"Dec 1, 2015",Brooklyn Bizarre,$0,$0
5486,87,"Aug 11, 2015",Alleluia! The Devil's Carnival,$0,$0
5485,86,"Jun 23, 2015",Crossroads,$0,$0
